In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import json
import re



res=requests.get('http://news.sina.com.cn/c/nd/2018-06-08/doc-ihcscwxa1809510.shtml')
res.encoding='utf-8'
soup=BeautifulSoup(res.text,'html.parser')    
title=soup.select('.main-title')[0].text
#timesource1=soup.select('.date-source')[0].text.split('\n')[1]    #获取时间
timesource=soup.select('.date-source span')[0].text         #获取时间
dt=datetime.strptime(timesource,'%Y年%m月%d日 %H:%M')
dt.strftime('%Y-%m-%d')
place=soup.select('.date-source a')[0].text    #获取新闻来源
article=[]                                   #获取文章内容
for p in soup.select('#article p')[:-1]:
    article.append(p.text.strip())
articleall=' '.join(article)
editor=soup.select('#article p')[-1].text.strip('责任编辑：')     #获取作者姓名
comments=requests.get('http://comment5.news.sina.com.cn/page/info?version=1&format=json&\
channel=gn&newsid=comos-hcscwxa1809510&group=undefined&compress=0&ie=utf-8&\
oe=utf-8&page=1&page_size=3&t_size=3&h_size=3&thread=1')                      
#print(comments.text)    
jd=json.loads(comments.text)         #用jason解析器
comment_num=jd['result']['count']['total']        #获得评论总数

In [ ]:
title

In [ ]:

#newsurl='http://news.sina.com.cn/c/nd/2018-06-08/doc-ihcscwxa1809510.shtml'

newsurl = 'https://news.sina.com.cn/c/zj/2019-06-12/doc-ihvhiqay5255597.shtml'
commenturl='http://comment5.news.sina.com.cn/page/info?version=1&format=json&channel=gn&newsid=comos-{}&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&h_size=3&thread=1'
def getcommentcounts(newsurl):            #获取评论数
    m=re.compile('doc-i(.*?).shtml').findall(newsurl)
    newsid=m[0]
    comments=requests.get(commenturl.format(newsid))
    jd=json.loads(comments.text)
    return jd['result']['count']['total']
 
def getnewsdetail(newsurl):                                        #获得单页的新闻内容
    result={}
    res=requests.get(newsurl)
    res.encoding='utf-8'
    soup=BeautifulSoup(res.text,'html.parser')
    result['title']=soup.select('.main-title')[0].text      #标题
    timesource=soup.select('.date-source span')[0].text  
    result['time']=datetime.strptime(timesource,'%Y年%m月%d日 %H:%M').strftime('%Y-%m-%d')              #时间
    result['place']=soup.select('.source')[0].text       #来源
    article=[]                                   #获取文章内容
    for p in soup.select('#article p')[:-1]:
        article.append(p.text.strip())
    articleall=' '.join(article)
    result['article']=articleall
    result['editor']=soup.select('#article p')[-1].text.strip('责任编辑：')     #获取作者姓名
    result['comment_num']=getcommentcounts(newsurl)
    return result

In [ ]:
getnewsdetail(newsurl)

In [ ]:
list_url = 'https://news.sina.com.cn/c/zj/2019-06-12/doc-ihvhiqay5255597.shtml'



In [ ]:
url='http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&cat_2==gdxw1||=gatxw||=zs-pl||=mtjj&level==1||=2&show_ext=1&show_all=1&show_num=22&tag=1&format=json&page={}&callback=newsloadercallback&_=1528548757769'
def parseListLinks(url):
    newsdetail=[]
    res=requests.get(url)
    print(res.text)
    jd=json.loads(res.text.lstrip('  newsloadercallback(').rstrip(');'))
    for ent in jd['result']['data']:
        newsdetail.append(getnewsdetail(ent['url']))
    return newsdetail

In [ ]:
news_total=[]
for i in range(1,2):
    newsurl=url.format(i)
    newsary=parseListLinks(newsurl)
    news_total.extend(newsary)

In [ ]:
import pandas as pd
df=pd.DataFrame(news_total)

In [ ]:
df

In [ ]:
news_total

In [ ]:

from bs4 import BeautifulSoup
from urllib import request
def download(title, url,m):
    req = request.Request(url)
    response = request.urlopen(req)
    response = response.read().decode('utf-8')
    soup = BeautifulSoup(response,'lxml')
    tag = soup.find('div',class_='article')
    if tag == None:
        return 0
    #print(type(tag))
    #print(tag.get_text())
    title = title.replace(':','')
    title = title.replace('"','')
    title = title.replace('|','')
    title = title.replace('/','')
    title = title.replace('\\','')
    title = title.replace('*','')
    title = title.replace('<','')
    title = title.replace('>','')
    title = title.replace('?','')
    #print(tag.get_text())
    filename = r'D:\code\python\spider_news\sina_news\sociaty\\' +title+'.txt'
    with open(filename,'w',encoding='utf8') as file_object:
        file_object.write('           ')
        file_object.write(title)
        file_object.write(tag.get_text())
    print('正在爬取第', m,'个新闻',title)
    return 0
if __name__ == '__main__':
    target_url = 'http://news.sina.com.cn/society/'
    req = request.Request(target_url)
    response = request.urlopen(req)
    response = response.read().decode('utf8')
    #print(response)
    soup = BeautifulSoup(response,'lxml')
    #print(soup.prettify())
    #file = open('d:\\test2.txt','w',encoding='utf8')
    #file.write(soup.prettify())
    y = 0
    for tag in soup.find_all('div',class_='news-item'):
        if tag.a != None:
            if len(tag.a.string) > 8:
                 #print(tag.a.string,tag.a.get('href'))
                 temp = tag.a.string
                 y += 1
                 download(temp,tag.a.get('href'),y)
 

In [20]:
import csv
import threading
import multiprocessing
import json
from urllib.parse import urlencode
from urllib.request import urlopen,Request,urlparse,build_opener,install_opener
from urllib.error import URLError,HTTPError

def html_download(url):

    headers = {'User-Agent': "User-Agent:Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)"}
    request = Request(url, headers=headers)
    try:
        html = urlopen(request).read().decode()
    except HTTPError as e:
        html = None
        print('[W] 下载出现服务器错误: %s' % e.reason)
        return None
    except URLError as e:
        html = None
        print("[E] 站点不可达: %s" % e.reason)
        return None
    return html

 
def api_info_manager(page):
    #http://api.roll.news.sina.com.cn/zt_list?channel=news&cat_1=gnxw&show_all=1&show_num=6000&tag=1&format=json
    comment_channel = [ 'gnxw', 'shxw','gjxw']
    for comment in comment_channel: 

        data= {
            'channel':'news',
            'cat_1':comment,
            'show_all':1,
            'show_num':5,
            'tag':1,
            'page':page,
            'format':'json'
            }
        dataformat = 'http://api.roll.news.sina.com.cn/zt_list?'+urlencode(data)
        response = html_download(dataformat)
        #print(response)
        json_results = json.loads(response,encoding='utf-8')['result']['data']

        for info_dict in json_results:
            print(info_dict)
            
            yield info_dict
 
 
fileheader = ['id','column','title','url','keywords','comment_channel','img','level','createtime','old_level','media_type','media_name']


def write_csv_header(fileheader):
    with open("新浪新闻.csv", "a", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fileheader)
        writer.writeheader()

    
def save_to_csv(result):
    with open("新浪新闻.csv", "a", encoding = "utf-8", newline="") as csvfile:
        print('正在写入csv文件中')
        writer = csv.DictWriter(csvfile, fieldnames=fileheader)
        writer.writerow(result)


def main(page):
    for res in api_info_manager(page):
        save_to_csv(res)

write_csv_header(fileheader)
main(1)

{'id': '1-1-36504939', 'column': '', 'title': '推进人工智能发展需各方联合行动', 'url': 'http://news.sina.com.cn/c/2018-11-20/doc-ihnyuqhi5168899.shtml', 'keywords': '人工智能,联合行动,肖立志', 'comment_channel': 'gn', 'img': '', 'level': '0', 'createtime': '1542722573', 'old_level': '3', 'media_type': '', 'media_name': '中国经济网'}
正在写入csv文件中
{'id': '1-1-36504938', 'column': '', 'title': '人民日报评论员：为世界经济拓展新的增长空间', 'url': 'http://news.sina.com.cn/o/2018-11-20/doc-ihmutuec2061859.shtml', 'keywords': '一带一路,世界经济,评论员', 'comment_channel': '', 'img': '', 'level': '0', 'createtime': '1542722528', 'old_level': '3', 'media_type': '', 'media_name': '人民日报'}
正在写入csv文件中
{'id': '1-1-36504936', 'column': '', 'title': '“银狐”国内告别战，球迷横幅上写着“里皮先生谢谢您”', 'url': 'http://news.sina.com.cn/c/2018-11-20/doc-ihmutuec2061390.shtml', 'keywords': '体育场,里皮,银狐', 'comment_channel': 'gn', 'img': '', 'level': '0', 'createtime': '1542722425', 'old_level': '3', 'media_type': '', 'media_name': '新京报'}
正在写入csv文件中
{'id': '1-1-36504932', 'column': '', 'title': '

In [ ]:
if __name__ == '__main__':
    #多线程  
    write_csv_header(fileheader)   
    pool = multiprocessing.Pool()  
    # 多进程  
    thread = threading.Thread(target=pool.map,args = (main,[x for x in range(1, 100)]))  
    thread.start()  
    thread.join()

In [10]:
import pandas as pd
f = open("新浪新闻.csv", encoding="utf-8")
df = pd.read_csv(f)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xba in position 224738: invalid start byte